# find-missing-pages-to-hand-label
SanjayKAroraPhD@gmail.com <br>
February 2019

## Description
Compare the output of the current labeled file against mongodb to see if there should be any additions

In [13]:
import pandas as pd
import pymongo
import pprint

In [64]:
MONGODB_DB = "FirmDB_20181226"
MONGODB_COLLECTION = "pages_ABOUT2"
CONNECTION_STRING = "mongodb://localhost"

client = pymongo.MongoClient(CONNECTION_STRING)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

ABOUT_DIR = '/Users/sarora/dev/EAGER/data/orgs/about/'
DATA_DIR = '/Users/sarora/dev/EAGER/data/orgs/parsed_page_output/'
TRAINING_PERCENT = .10

PHRASE_LENGTH = 60
MIN_PARA_LEN = 5

pp = pprint.PrettyPrinter()

In [83]:
df = pd.read_csv('/Users/sarora/dev/EAGER/data/orgs/about/about_pages_labeled_v3.csv')
df.head()

,firm_name,url,about_lbl,mgmt_lbl,partners_lbl,ip_lbl,about_agg_lbl,gid
0,3M Innovative Properties Company,https://www.3m.com/,NaN,NaN,NaN,NaN,0,1
1,3M Innovative Properties Company,https://www.3m.com/3M/en_US/company-us/site-map/,NaN,NaN,NaN,NaN,0,1
2,3M Innovative Properties Company,https://www.3m.com/3M/en_US/company-us/about-3m/,1.0,NaN,NaN,NaN,1,1
3,3M Innovative Properties Company,https://www.3m.com/3M/en_US/home-improvement-us/,NaN,NaN,NaN,NaN,0,1
4,3M Innovative Properties Company,https://www.3m.com/3M/en_US/company-us/SDS-sea...,NaN,NaN,NaN,NaN,0,1


In [84]:
labeled_firm_names = list(set(df['firm_name']))
labeled_urls = list(df['url'])

print (labeled_firm_names[0:5])
print (labeled_urls[0:5])

['SAE Magnetics (HK) Ltd.', 'Microchips Biotech', 'Allison Transmission', 'Greatbatch Ltd.', 'iNanoBio LLC']
['https://www.3m.com/', 'https://www.3m.com/3M/en_US/company-us/site-map/', 'https://www.3m.com/3M/en_US/company-us/about-3m/', 'https://www.3m.com/3M/en_US/home-improvement-us/', 'https://www.3m.com/3M/en_US/company-us/SDS-search/']


In [86]:
results = col.find ({}, {'url': 1, 'firm_name': 1})
url_to_firm_name_dict = {}

for result in results: 
    url = result['url'][0]
    firm_name = result['firm_name'][0]
    url_to_firm_name_dict[url] = firm_name

In [88]:
missing = []
for mongo_url, mongo_firm_name in url_to_firm_name_dict.items(): 
    # print (mongo_firm_name)
    if mongo_firm_name not in labeled_firm_names:
        continue
    if mongo_url not in labeled_urls: 
        missing.append ([mongo_firm_name, mongo_url])

In [95]:
print(len(missing))
# pp.pprint (missing)

df_missing = pd.DataFrame(missing, columns=['firm_name', 'url'])
df_missing.to_csv('/Users/sarora/dev/EAGER/data/orgs/about/about_pages_labeled_v3-missing.csv', index=False)

834
